In [8]:
!pip install torch transformers accelerate bitsandbytes qwen-vl-utils langchain chromadb sentence-transformers -q
!pip install torchvision -q
!pip install langchain-community langchain-huggingface chromadb sentence-transformers -q
!pip install langchain langchain-core -q
!pip install langchain-community -q
!pip install --upgrade torch torchvision --extra-index-url https://download.pytorch.org/whl/cu121 -q

In [1]:
import torch
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor,BitsAndBytesConfig
from PIL import Image
import requests
from io import BytesIO

In [2]:

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Çalışma Ortamı: {device.upper()}")
if device == "cuda":
    print(f"GPU Modeli: {torch.cuda.get_device_name(0)}")
    print(f"VRAM Bilgisi: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("UYARI: CPU kullanıyorsunuz. Model çok yavaş çalışacaktır!")

Çalışma Ortamı: CUDA
GPU Modeli: NVIDIA GeForce RTX 4060
VRAM Bilgisi: 8.59 GB


In [1]:
import accelerate
import torch
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
from PIL import Image
import requests
from io import BytesIO

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Cihaz: {device}")

model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct",
    dtype=torch.bfloat16,
    device_map="auto",
    attn_implementation="sdpa"
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")

Cihaz: cuda


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.


In [3]:
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document 
import torch
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
from PIL import Image
import requests
from io import BytesIO
import json

with open("diyet_bilgileri.json", "r", encoding="utf-8") as f:
    diyet_bilgileri_json = json.load(f)

diyet_bilgileri = [f"{item['food']}: {item['calories']} kcal. {item['description']}" for item in diyet_bilgileri_json]

docs = [Document(page_content=text) for text in diyet_bilgileri]

vector_db = Chroma.from_documents(
    documents=docs, 
    collection_name="kalori_cetveli_v3"
)


docs = [Document(page_content=text) for text in diyet_bilgileri]

vector_db = Chroma.from_documents(
    documents=docs, 
    collection_name="kalori_cetveli_v3"
)

def bilgi_getir(sorgu):
    sonuclar = vector_db.similarity_search(sorgu, k=2)
    return "\n".join([doc.page_content for doc in sonuclar])

print(" Veritabanı Hazır.")

print("2. Model yükleniyor...")

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Cihaz: {device.upper()}")

model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct",
    torch_dtype=torch.bfloat16, 
    device_map="auto",
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")

print(" Model Yüklendi.")

print("3. Analiz yapılıyor...")

image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/0/0b/RedDot_Burger.jpg/800px-RedDot_Burger.jpg"
user_question = "Bu tabakta ne var ve tahmini kalorisi nedir?"

response = requests.get(image_url, headers={'User-Agent': 'Mozilla/5.0'})
image = Image.open(BytesIO(response.content))

rag_context = bilgi_getir("hamburger ve patates kalori") 
print(f"\n Veritabanından Bulunan Bilgi:\n{rag_context}\n")

system_prompt = """Sen uzman bir diyetisyensin. Görselleri analiz ederken şu adımları izle:
1. Tabağı analiz et.
2. Veritabanından gelen bilgileri kontrol et.
3. Sonucu net bir liste halinde ver.

ÖRNEK 1:
Kullanıcı Resmi: [Salata Resmi]
Context Bilgisi: Sezar salata 400 kalori.
Analiz: Tabakta yeşillik ve tavuk var. Soslu görünüyor.
Cevap: Tavuklu Sezar Salata (400 kcal). Sos içerdiği için yağ oranı yüksektir.

ŞİMDİ SIRA SENDE:
"""

final_text_prompt = f"""
{system_prompt}

BAĞLAM BİLGİSİ (Veritabanından):
{rag_context}

SORU:
{user_question}
"""

conversation = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": final_text_prompt},
        ],
    }
]

text_prompt = processor.apply_chat_template(conversation, tokenize=False, add_generation_prompt=True)
inputs = processor(text=[text_prompt], images=[image], return_tensors="pt", padding=True).to(device)

output_ids = model.generate(**inputs, max_new_tokens=200)
generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, output_ids)]
output_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

print("-" * 30)
print("MODELİN CEVABI:")
print(output_text)

C:\Users\Nazar\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [01:02<00:00, 1.33MiB/s]
`torch_dtype` is deprecated! Use `dtype` instead!


 Veritabanı Hazır.
2. Model yükleniyor...
Cihaz: CUDA


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


 Model Yüklendi.
3. Analiz yapılıyor...

 Veritabanından Bulunan Bilgi:
Hamburger: 550 kcal. Orta boy hamburger, yüksek karbonhidrat.
Hamburger: 550 kcal. Orta boy hamburger, yüksek karbonhidrat.

------------------------------
MODELİN CEVABI:
Tabakta hamburger, domates, salatalık ve patates tazeği bulunuyor. Hamburgerin kalorisi 550 kcal'dir.
